In [1]:
#Import all the nessacary libraries to run the pose estimator

#Import mediapipe to be used for the model
import mediapipe as mp
#Import opencv for rendaring and drawing capabilities
import cv2

import numpy as np #Handle numpy arrays
import pandas as pd #Handle tabular data
import os #Handle folder structure
import pickle #Save and oad ML model

In [9]:
#Import the model from the binary file
with open('rf_model.pkl', 'rb') as f:
    model = pickle.load(f)
    print("Model Loaded")
    
print(model)

Model Loaded


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [7]:
#Display the results of the prediction done by the model

holistic_model = mp.solutions.holistic 

#Connect the test video from the device
sample_video = cv2.VideoCapture('datasets/test/valid/5.valid.mp4')

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        status, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if status == False:
            print("Done")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        
         #Predict the coordinates of the landmarks (resulrs screen)
        try:
            #Extracting all the landmarks of the pose as an array
            pose_landmarks_array = result_frame.pose_landmarks.landmark
            #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
            pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks_array]).flatten())

            #Pass the numpy array into a data frame
            features = pd.DataFrame([pose_landmarks_nparray])
            
            #Store the top class of the prediction
            pose_class_status = model.predict(features.values)[0]
            #Store the probability of the prediction
            pose_class_status_prob = model.predict_proba(features.values)[0]
            
            print("Class:", pose_class_status)
            print(pose_class_status_prob)
            
            #Set a rectangle box to display the results of the prediction in the video frame
            #rectangle(container, top_coord, bottom_coord, color, line_thickness)
            cv2.rectangle(bgr_frame, (0,0), (250, 60), (245, 117, 16), -1)
            
            #Display the class label inside the rectangle box
            cv2.putText(bgr_frame, 'Class'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract =and display the top class of the prediction
            cv2.putText(bgr_frame, pose_class_status.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display the class probability inside the rectangle box
            cv2.putText(bgr_frame, 'Probability'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and dispthe maximum probability
            cv2.putText(bgr_frame, str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
 
        except:
            pass
                        
        #Display the frames    
        cv2.imshow('Results Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

sample_video.release()
cv2.destroyAllWindows()

Class: Incorrect
[0.41 0.59]
Class: Incorrect
[0.42 0.58]
Class: Incorrect
[0.42 0.58]
Class: Incorrect
[0.43 0.57]
Class: Incorrect
[0.46 0.54]
Class: Incorrect
[0.42 0.58]
Class: Incorrect
[0.41 0.59]
Class: Incorrect
[0.41 0.59]
Class: Incorrect
[0.42 0.58]
Class: Incorrect
[0.44 0.56]
Class: Incorrect
[0.42 0.58]
Class: Incorrect
[0.44 0.56]
Class: Incorrect
[0.44 0.56]
Class: Incorrect
[0.44 0.56]
Class: Incorrect
[0.44 0.56]
Class: Incorrect
[0.46 0.54]
Class: Correct
[0.57 0.43]
Class: Correct
[0.6 0.4]
Class: Correct
[0.61 0.39]
Class: Correct
[0.59 0.41]
Class: Correct
[0.54 0.46]
Class: Correct
[0.6 0.4]
Class: Correct
[0.59 0.41]
Class: Correct
[0.58 0.42]
Class: Correct
[0.53 0.47]
Class: Correct
[0.53 0.47]
Class: Correct
[0.53 0.47]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
Class: Correct
[0.52 0.48]
